# Sample notebook for using Copernicus Toolbox 

This notebook is an to download ocean model data to d

In [2]:
import xarray as xr
import copernicusmarine
import copernicusmarine as cm
from pprint import pprint
import json
import matplotlib
import cartopy
cm.__version__

'2.2.1'

In [3]:
# Download the catalog for the MERCATOR global reanalysis runs
catalogue_001_024 = copernicusmarine.describe(product_id="GLOBAL_ANALYSISFORECAST_PHY_001_024", 
                                              disable_progress_bar=True,
                                              
                                              )

In [4]:
# Create a list of datasets we're interested in
# Filtering for the daily files
# Filtering for "cur" (currents), "so" (salinity) and "theta0" (temperature)
#
# This is horrible, there must be a more elegant pythonic way to do this

# make a dictionary
dataset_list = {}
for dataset in catalogue_001_024.products[0].datasets:
    
    dataset_id = dataset.dataset_id
    if dataset_id.count("P1D")>0:
        print(dataset_id)
        if dataset_id.count("-cur_"):
            dataset_list[dataset_id] = ["uo","vo"]
        if dataset_id.count("-so"):
            dataset_list[dataset_id] = ["so"]
        if dataset_id.count("-thetao"):
            dataset_list[dataset_id] = ["thetao"]
print(dataset_list)

cmems_mod_glo_phy-cur_anfc_0.083deg_P1D-m
cmems_mod_glo_phy-so_anfc_0.083deg_P1D-m
cmems_mod_glo_phy-thetao_anfc_0.083deg_P1D-m
cmems_mod_glo_phy-wcur_anfc_0.083deg_P1D-m
cmems_mod_glo_phy_anfc_0.083deg-sst-anomaly_P1D-m
cmems_mod_glo_phy_anfc_0.083deg_P1D-m
{'cmems_mod_glo_phy-cur_anfc_0.083deg_P1D-m': ['uo', 'vo'], 'cmems_mod_glo_phy-so_anfc_0.083deg_P1D-m': ['so'], 'cmems_mod_glo_phy-thetao_anfc_0.083deg_P1D-m': ['thetao']}


In [5]:
# Now, Loop through all the datasets of interest, download the individual netCDFs for the subset region
# (in space and time)

# You will need a copernicus account, and need to enter the credentials when you run this cell
# each time you take a subset

# Only need to run this cell once to download the data


# TODO: Figure out how to use an SSH key or other to keep from having to type in credentials


for dataset_id in dataset_list.keys():
    #dataset_id = "cmems_mod_glo_phy_anfc_0.083deg_P1D-m"   # 3D daily
    vars_3d = dataset_list[dataset_id]                  # east vel, north vel, potential temp, salinity

    # cm.subset(
    #     dataset_id=dataset_id,
    #     variables=vars_3d,
    #     minimum_longitude=-70,  # west
    #     maximum_longitude=-65,  # east
    #     minimum_latitude=  30.0,   # south
    #     maximum_latitude=  45.0,   # north
    #     start_datetime="2025-07-01T00:00:00",
    #     end_datetime="2025-07-03T00:00:00",
    #     minimum_depth=0,           # meters (0 = surface)
    #     maximum_depth=1000,         # crop to upper 200 m
    #     output_filename=f"{dataset_id}_subset.nc",
    #     output_directory="copernicus-data",
    #     file_format="netcdf",      # or "zarr"
    #     netcdf_compression_level=4 # optional tradeoff size vs speed
    # )

In [6]:
cur_ds = xr.open_dataset("copernicus-data/cmems_mod_glo_phy-cur_anfc_0.083deg_P1D-m_subset.nc")
so_ds = xr.open_dataset("copernicus-data/cmems_mod_glo_phy-so_anfc_0.083deg_P1D-m_subset.nc")
thetao_ds = xr.open_dataset("copernicus-data/cmems_mod_glo_phy-thetao_anfc_0.083deg_P1D-m_subset.nc")

In [7]:
all_ds = cur_ds
all_ds['thetao'] = thetao_ds['thetao']
all_ds['so'] = so_ds['so']
all_ds_mean = all_ds.mean(dim='time')
all_ds_mean

<xarray.Dataset> Size: 6MB
Dimensions:    (depth: 35, latitude: 181, longitude: 61)
Coordinates:
  * depth      (depth) float32 140B 0.494 1.541 2.646 ... 643.6 763.3 902.3
  * latitude   (latitude) float32 724B 30.0 30.08 30.17 ... 44.83 44.92 45.0
  * longitude  (longitude) float32 244B -70.0 -69.92 -69.83 ... -65.08 -65.0
Data variables:
    uo         (depth, latitude, longitude) float32 2MB 0.0193 0.04666 ... nan
    vo         (depth, latitude, longitude) float32 2MB 0.1814 0.1647 ... nan
    thetao     (depth, latitude, longitude) float32 2MB 29.27 29.29 ... nan nan
    so         (depth, latitude, longitude) float32 2MB 37.0 37.01 ... nan nan

In [8]:
all_ds_mean.to_netcdf('copernicus-data/copernicus-subset-jul-01-03-2025')